# Agent using logic fuction OpenAPI

In [25]:
import os
import jsonref
from typing import Set
from dotenv import load_dotenv
import json
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails

load_dotenv()

True

In [26]:
try:
    with open("./openapi_get_user_accounts.json", "r") as f:
        openapi_get_user_accounts_logic_app = jsonref.loads(f.read())
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except FileNotFoundError:
    print("File not found. Please check the file path.")

In [27]:
# 1. Authenticate with the Azure AI project service
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ.get("PROJECT_CONNECTION_STRING"),
)

# 2. Add functions to the agent
auth = OpenApiAnonymousAuthDetails()

In [28]:
openapi_tool = OpenApiTool(
    name="record_account_logic_app",
    spec=openapi_get_user_accounts_logic_app,
    description="Registra un medio transaccional en la tabla de cuentas",
    auth=auth,
    default_parameters=["format"],
    )

In [29]:
openapi_tool.definitions

[{'type': 'openapi', 'openapi': {'name': 'record_account_logic_app', 'description': 'Registra un medio transaccional en la tabla de cuentas', 'spec': {'openapi': '3.0.0', 'info': {'title': 'get_user_accounts', 'description': 'Retrieves a list of accounts for the user.', 'version': '1.0.0.0'}, 'servers': [{'url': 'https://prod-06.eastus2.logic.azure.com/workflows/75a8695be7b649d7aa6fc22ef9696423/triggers'}], 'paths': {'/When_a_HTTP_request_is_received/paths/invoke': {'post': {'summary': 'Get User Accounts', 'description': 'Retrieves a list of accounts for the user.', 'operationId': 'GetUserAccounts', 'parameters': [{'name': 'api-version', 'in': 'query', 'description': 'The service API version.', 'required': True, 'schema': {'type': 'string', 'default': '2016-10-01'}}, {'name': 'sp', 'in': 'query', 'description': "The permissions; generally 'read' or 'write'.", 'required': True, 'schema': {'type': 'string', 'default': '/triggers/When_a_HTTP_request_is_received/run'}}, {'name': 'sv', 'in'

In [30]:
with project_client:
        
        # Create agent with explicit instructions
        agent_instructions = """
        You are a specialized agent for retrieving user account information.
        
        IMPORTANT: You have access to the following function:
        - record_account_logic_app(user_id: int) -> str
        This function takes a user ID as an integer and returns account information as a JSON string.
        
        When asked about user accounts:
        1. Extract the user ID from the request
        2. Call record_account_logic_app with the user ID parameter
        3. Return the account information in a readable format
        
        Always confirm when you've successfully retrieved the information.
        """

        agent = project_client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            name="Agent2-Inputs",
            instructions=agent_instructions,
            tools=openapi_tool.definitions,
        )
        print(f"Created agent, ID: {agent.id}")
        
        # Create a thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create a message in the thread
        # Test account creation
        # msgcontent = "Quiero registrar una cuenta que se llama Davivienda, es de ahorros. Es mi cuenta de nómina. Mi ID de usuario es 1."
        # Test category creation
        msgcontent = "Hello, please get the user accounts for user ID 1."
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=msgcontent,
        )
        print(f"Created message, ID: {message.id}")

        # Create and process an agent run in the thread
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed" or run.status == "incomplete":
            print(f"Run failed: {run.last_error}")

        # Delete the agent when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print(f"Messages: {messages}")

Created agent, ID: asst_w2TWqzP21nzDGGVxX9BriBXO
Created thread, ID: thread_474cvWO2LNxXgYaRpd4JeXQr
Created thread, ID: thread_474cvWO2LNxXgYaRpd4JeXQr
Created message, ID: msg_zlu4dUQ5Qso6pfuusMxaSmnY
Created message, ID: msg_zlu4dUQ5Qso6pfuusMxaSmnY
Run finished with status: RunStatus.COMPLETED
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_LdzoIUBHeachCwuYLnlZO601', 'object': 'thread.message', 'created_at': 1745195190, 'assistant_id': 'asst_w2TWqzP21nzDGGVxX9BriBXO', 'thread_id': 'thread_474cvWO2LNxXgYaRpd4JeXQr', 'run_id': 'run_bXpRRlf3QCi4eDQThadTMwbB', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'I successfully retrieved the account information for user ID 1. Here are the details of the accounts:\n\n1. **Checking Account** \n   - Type: Bank\n2. **Savings Account** \n   - Type: Bank\n3. **Credit Card** \n   - Type: Credit\n4. **Ahorros Davivienda** \n   - Type: Activo\n5. **Davivienda** \n   - Type: Activo\n